##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 将 TensorFlow 1 代码迁移到 TensorFlow 2

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/guide/migrate">     <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">     在 TensorFlow.org 上查看</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/migrate.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/migrate.ipynb">     <img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">     在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/guide/migrate.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table>

本指南面向使用低级 TensorFlow API 的用户。如果您正在使用高级 API (`tf.keras`)，可能无需或仅需对您的代码执行少量操作，便可以让代码完全兼容 TensorFlow 2.x：

- 查看您的[优化器的默认学习率](#keras_optimizer_lr)。
- 请注意，记录指标时使用的“名称”[可能已发生变化](#keras_metric_names)。

在 TensorFlow 2.x 中，1.X 的代码不经修改也许还可以运行（[除了 contrib](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md)）：

```python
import tensorflow.compat.v1 as tf tf.disable_v2_behavior()
```

但是，这样做无法让您利用到我们对 TensorFlow 2.0 做出的许多改进。本指南将帮助您升级代码，让代码更简洁、性能更出色、维护更轻松。

## 自动转换脚本

尝试实现本指南中介绍的变更之前，第一步是尝试运行[升级脚本](./upgrade.md)。

这是将您的代码升级到 TensorFlow 2.x 的初始步骤，但您的代码不会因此便符合 2.x 的惯用标准。您的代码仍然可以使用 `tf.compat.v1` 端点来访问占位符、会话、集合以及其他 1.x 风格的功能。

## 顶级行为变更

如果您的代码使用 `tf.compat.v1.disable_v2_behavior` 在 TensorFlow 2.x 中运行，仍存在您需要处理的全局行为变更。重大变更包括：

- *Eager Execution，`v1.enable_eager_execution()`*：任何隐式使用 `tf.Graph` 的代码都会执行失败。确保将此代码封装到 `with tf.Graph().as_default()` 上下文中。

- *资源变量，`v1.enable_resource_variables()`*：有些代码会依赖于由 TensorFlow 引用变量带来的不确定行为。资源变量在写入时处于锁定状态，因此可提供更直观的一致性保证。

    - 这可能会更改极端情况下的行为。
    - 这可能会创建额外的副本以及使用更多的内存。
    - 可以通过将 `use_resource=False` 传递给 `tf.Variable` 构造函数来停用资源变量。

- *Tensor 形状，`v1.enable_v2_tensorshape()`*：TensorFlow 2.x 简化了张量形状的行为。您可以使用 `t.shape[0]` 代替 `t.shape[0].value`。这些变更不大，立即修正它们很有意义。有关示例，请参阅 [TensorShape](#tensorshape) 部分。

- *控制流，`v1.enable_control_flow_v2()`*：TensorFlow 2.x 的控制流实现已得到简化，因此会产生不同的计算图表示。如有任何问题，请[提交错误](https://github.com/tensorflow/tensorflow/issues)。

## 为 TensorFlow 2.x 创建代码

本文会详细介绍将 TensorFlow 1.x 代码转换为 TensorFlow 2.0 代码的几个示例。这些变更可让您的代码充分利用性能优化和简化的 API 调用。

在每种情况下，模式为：

### 1. 替换 `v1.Session.run` 调用

每个 `v1.Session.run` 调用都应该替换为 Python 函数。

- `feed_dict` 和 `v1.placeholder` 成为函数参数。
- `fetches` 成为函数的返回值。
- 在转换期间，Eager Execution 使用 `pdb` 等标准 Python 工具让调试变得简单。

之后，添加一个 `tf.function` 装饰器，这样可以使其高效地在计算图中运行。要详细了解其工作原理，请参阅 [Autograph 指南](function.ipynb)。

请注意：

- 与 `v1.Session.run` 不同，`tf.function`  具有固定的返回签名，并且始终返回所有输出。如果这样会导致性能问题，建议创建两个单独的函数。

- 不需要进行 `tf.control_dependencies` 或类似的运算：`tf.function` 会像事先编写好一样按顺序运行。例如，`tf.Variable` 赋值和 `tf.assert` 会自动执行。

[转换模型部分](#converting_models)包含此转换过程的一个有效示例。


### 2. 使用 Python 对象跟踪变量和损失

在 TensorFlow 2.x 中，强烈建议不要使用基于名称的变量跟踪。请使用 Python 对象跟踪变量。

使用 `tf.Variable` 而不是 `v1.get_variable`。

每个 `v1.variable_scope` 都应当转换为 Python 对象。通常是下列对象之一：

- `tf.keras.layers.Layer`
- `tf.keras.Model`
- `tf.Module`

如果您需要聚合变量列表（如 `tf.Graph.get_collection(tf.GraphKeys.VARIABLES)`），请使用 `Layer` 和 `Model` 对象的 `.variables` 和 `.trainable_variables` 特性。

这些 `Layer` 和 `Model` 类会实现多种其他属性，因此不需要全局集合。它们的 `.losses` 属性可以代替 `tf.GraphKeys.LOSSES` 集合。

请参阅 [Keras 指南](keras.ipynb)，了解更多详细信息。

警告：许多 `tf.compat.v1` 符号会以隐式方式使用全局集合。


### 3. 升级您的训练循环

使用适合您的用例的最高级 API。推荐使用 `tf.keras.Model.fit` 构建您自己的训练循环。

这些高级函数负责管理许多您自己编写训练循环时容易遗漏的低级细节。例如，这些函数会自动收集正则化损失，并在调用模型时设置 `training=True` 参数。


### 4. 升级您的数据输入流水线

使用 `tf.data` 数据集进行数据输入。这些对象高效、表达性强，并且可与 TensorFlow 很好地集成。

它们可以直接传递到 `tf.keras.Model.fit` 方法。

```python
model.fit(dataset, epochs=5)
```

它们可以直接通过标准 Python 进行迭代：

```python
for example_batch, label_batch in dataset:     break
```


### 5. 迁移 `compat.v1` 符号

`tf.compat.v1` 模块包含完整的 TensorFlow 1.x API，且具有其原始语义。

如果此类转换安全，[TensorFlow 2.x 升级脚本](upgrade.ipynb)会将符号转换为 v2 中的对应符号，即脚本能够确认 TensorFlow 2.x 版本的行为完全等效（例如，脚本判断两者是同一个函数，因此将 `v1.arg_max` 重命名为 `tf.argmax`）。

升级脚本运行完成后，会留下一段代码，其中很可能多次出现 `compat.v1`。建议逐行检查代码，手动将其转换为 v2 版本中的对应符号（如果有对应的符号，会在日志中提及）。

## 转换模型

### 低级变量和运算符执行

低级 API 使用的示例包括：

- 使用变量作用域控制重用。

- 使用 `v1.get_variable` 创建变量。

- 以显式方式访问集合。

- 通过以下方法以隐式方式访问集合：

    - `v1.global_variables`
    - `v1.losses.get_regularization_loss`

- 使用 `v1.placeholder` 设置计算图输入。

- 使用 `Session.run` 执行计算图。

- 手动初始化变量。

#### 转换前

这些模式在使用 TensorFlow 1.x 的代码中如下所示：


In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as v1

import tensorflow_datasets as tfds

In [ ]:
g = v1.Graph()

with g.as_default():
  in_a = v1.placeholder(dtype=v1.float32, shape=(2))
  in_b = v1.placeholder(dtype=v1.float32, shape=(2))

  def forward(x):
    with v1.variable_scope("matmul", reuse=v1.AUTO_REUSE):
      W = v1.get_variable("W", initializer=v1.ones(shape=(2,2)),
                          regularizer=lambda x:tf.reduce_mean(x**2))
      b = v1.get_variable("b", initializer=v1.zeros(shape=(2)))
      return W * x + b

  out_a = forward(in_a)
  out_b = forward(in_b)
  reg_loss=v1.losses.get_regularization_loss(scope="matmul")

with v1.Session(graph=g) as sess:
  sess.run(v1.global_variables_initializer())
  outs = sess.run([out_a, out_b, reg_loss],
      	        feed_dict={in_a: [1, 0], in_b: [0, 1]})

print(outs[0])
print()
print(outs[1])
print()
print(outs[2])

#### 转换后

在转换后的代码中：

- 变量均为本地 Python 对象。
- `forward` 函数仍然定义计算。
-  `Session.run` 调用替换为对  `forward` 的调用。
- 可以添加可选的 `tf.function` 装饰器来提高性能。
- 可以手动计算正则化，无需引用任何全局集合。
- **没有使用会话或占位符**。

In [ ]:
W = tf.Variable(tf.ones(shape=(2,2)), name="W")
b = tf.Variable(tf.zeros(shape=(2)), name="b")

@tf.function
def forward(x):
  return W * x + b

out_a = forward([1,0])
print(out_a)

In [ ]:
out_b = forward([0,1])

regularizer = tf.keras.regularizers.l2(0.04)
reg_loss=regularizer(W)

### 基于 `tf.layers` 的模型

使用 `v1.layers` 模块来包含依赖于 `v1.variable_scope` 的层函数以定义和重用变量。

#### 转换前


In [ ]:
def model(x, training, scope='model'):
  with v1.variable_scope(scope, reuse=v1.AUTO_REUSE):
    x = v1.layers.conv2d(x, 32, 3, activation=v1.nn.relu,
          kernel_regularizer=lambda x:0.004*tf.reduce_mean(x**2))
    x = v1.layers.max_pooling2d(x, (2, 2), 1)
    x = v1.layers.flatten(x)
    x = v1.layers.dropout(x, 0.1, training=training)
    x = v1.layers.dense(x, 64, activation=v1.nn.relu)
    x = v1.layers.batch_normalization(x, training=training)
    x = v1.layers.dense(x, 10)
    return x

In [ ]:
train_data = tf.ones(shape=(1, 28, 28, 1))
test_data = tf.ones(shape=(1, 28, 28, 1))

train_out = model(train_data, training=True)
test_out = model(test_data, training=False)

print(train_out)
print()
print(test_out)

#### 转换后

- 这一简单的层堆栈可恰好置于 `tf.keras.Sequential` 中（要了解更复杂的模型，请参阅[自定义层和模型](keras/custom_layers_and_models.ipynb)以及[函数式 API](keras/functional.ipynb) 指南）。
- 模型可以跟踪变量和正则化损失。
- 转换是一对一的，因为有一个从 `v1.layers` 到 `tf.keras.layers` 的直接映射。

大多数参数都保持不变。但要注意区别：

- 在 `training` 参数运行时，模型会将其传递到每个层。
- 原始 `model` 函数的第一个参数（输入 `x`）会消失。这是因为对象层会将构建模型与调用模型分开。

另请注意：

- 如果您使用的是 `tf.contrib` 中的正则化器或初始值设定项，则与其他正则化器相比，会有更多参数变更。
- 代码不再写入集合，`v1.losses.get_regularization_loss` 之类的函数不再返回这些值，因此很可能破坏您的训练循环。

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.04),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10)
])

train_data = tf.ones(shape=(1, 28, 28, 1))
test_data = tf.ones(shape=(1, 28, 28, 1))

In [ ]:
train_out = model(train_data, training=True)
print(train_out)

In [ ]:
test_out = model(test_data, training=False)
print(test_out)

In [ ]:
# Here are all the trainable variables.
len(model.trainable_variables)

In [ ]:
# Here is the regularization loss.
model.losses

### 混合变量和 `v1.layers`


现有代码经常将低级 TensorFlow 1.x 变量和运算与更高级的 `v1.layers` 混合。

#### 转换前


In [ ]:
def model(x, training, scope='model'):
  with v1.variable_scope(scope, reuse=v1.AUTO_REUSE):
    W = v1.get_variable(
      "W", dtype=v1.float32,
      initializer=v1.ones(shape=x.shape),
      regularizer=lambda x:0.004*tf.reduce_mean(x**2),
      trainable=True)
    if training:
      x = x + W
    else:
      x = x + W * 0.5
    x = v1.layers.conv2d(x, 32, 3, activation=tf.nn.relu)
    x = v1.layers.max_pooling2d(x, (2, 2), 1)
    x = v1.layers.flatten(x)
    return x

train_out = model(train_data, training=True)
test_out = model(test_data, training=False)

#### 转换后

要转换此代码，请遵循将层映射到层的模式，如上例所示。

通用模式为：

- 在 `__init__` 中收集层参数。
- 在 `build` 中构建变量。
- 在 `call`  中执行计算，并返回结果。

`v1.variable_scope` 本质上是其自身的层。因此将其重写为 `tf.keras.layers.Layer`。请参阅[通过子类化添加新层和模型](keras/custom_layers_and_models.ipynb)指南以了解详情。

In [ ]:
# Create a custom layer for part of the model
class CustomLayer(tf.keras.layers.Layer):
  def __init__(self, *args, **kwargs):
    super(CustomLayer, self).__init__(*args, **kwargs)

  def build(self, input_shape):
    self.w = self.add_weight(
        shape=input_shape[1:],
        dtype=tf.float32,
        initializer=tf.keras.initializers.ones(),
        regularizer=tf.keras.regularizers.l2(0.02),
        trainable=True)

  # Call method will sometimes get used in graph mode,
  # training will get turned into a tensor
  @tf.function
  def call(self, inputs, training=None):
    if training:
      return inputs + self.w
    else:
      return inputs + self.w * 0.5

In [ ]:
custom_layer = CustomLayer()
print(custom_layer([1]).numpy())
print(custom_layer([1], training=True).numpy())

In [ ]:
train_data = tf.ones(shape=(1, 28, 28, 1))
test_data = tf.ones(shape=(1, 28, 28, 1))

# Build the model including the custom layer
model = tf.keras.Sequential([
    CustomLayer(input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
])

train_out = model(train_data, training=True)
test_out = model(test_data, training=False)


需要注意的一些事项：

- 子类化 Keras 模型和层需要在 v1 计算图（无自动控制依赖项）中以 Eager 模式运行：

    - 将 `call` 封装到 `tf.function` 中以获得 Autograph 和自动控制依赖项。

- 不要忘记接受 `call` 的 `training` 参数:

    - 有时是 `tf.Tensor`
    - 有时是 Python 布尔值

- 使用 `self.add_weight 在构造函数或 <code>Model.build</code> 中创建模型变量：

    - 在 `Model.build` 中，您可以访问输入形状，因此可以创建具有匹配形状的权重
    - 使用 `tf.keras.layers.Layer.add_weight` 后，Keras 可以跟踪变量和正则化损失

- 不要在您的对象中保留 `tf.Tensors`:

    - 它们可能会在 `tf.function` 或 Eager 上下文中创建，并且这些张量的行为有所不同
    - 使用 `tf.Variable` 获取状态，它们在两种上下文中始终可用
    - `tf.Tensors` 只适用于中间值

### 关于 Slim 和 contrib.layers 的注意事项

大量的旧 TensorFlow 1.x 代码使用 [Slim](https://ai.googleblog.com/2016/08/tf-slim-high-level-library-to-define.html) 库，后者使用 TensorFlow 1.x 打包为 `tf.contrib.layers`。作为一个 `contrib` 模块，它在 TensorFlow 2.x 中，甚至是 `tf.compat.v1` 中，都将不再可用。将使用 Slim 的代码转换到 TensorFlow 2.x 比转换使用 `v1.layers` 的仓库涉及的改动更多。事实上，最好先将您的 Slim 代码转换到 `v1.layers`，然后再转换到 Keras。

- 移除 `arg_scopes`，所有参数均需要为显式
- 如果要使用，请将 `normalizer_fn` 和 `activation_fn` 拆分到其自己的层中
- 可分离的卷积层映射至一个或多个不同的 Keras 层（深度、逐点和可分离 Keras 层）
- Slim 与 `v1.layers` 有不同的参数名和默认值。
- 某些参数具有不同的比例
- 如果您使用 Slim 预训练模型，可以尝试来自 `tf.keras.applications` 的 Keras 预训练模型或从原始 Slim 代码导出的 [TF Hub](https://tfhub.dev/s?tf-version=tf2&q=slim) TensorFlow 2.x SavedModel。

一些 `tf.contrib` 层可能没有移至核心 TensorFlow，相反，它们移至 [TensorFlow Addons 软件包](https://tensorflow.google.cn/addons/overview)。


## 训练

您可以通过多种方式将数据提供给 `tf.keras` 模型。这些方式接受 Python 生成器和 Numpy 数组作为输入。

推荐使用 `tf.data` 软件包将数据提供给模型，其中包含一组用于处理数据的高性能类。

如果您仍在使用 `tf.queue`，则现在仅支持将它们作为数据结构，而不能作为输入流水线。

### 使用 TensorFlow Datasets

[TensorFlow Datasets](https://tensorflow.org/datasets) 软件包 (`tfds`) 中包含用于将预定义数据集作为 `tf.data.Dataset` 对象加载的实用工具。

对于此示例，您可以使用 `tfds` 加载 MNIST 数据集：

In [ ]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

随后准备用于训练的数据：

- 重新缩放每个图像。
- 打乱样本的顺序。
- 收集批量图像和标签。


In [ ]:
BUFFER_SIZE = 10 # Use a much larger value for real code.
BATCH_SIZE = 64
NUM_EPOCHS = 5


def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

为了使样本简短，将数据集修剪为仅返回 5 个批次：

In [ ]:
train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_data = mnist_test.map(scale).batch(BATCH_SIZE)

STEPS_PER_EPOCH = 5

train_data = train_data.take(STEPS_PER_EPOCH)
test_data = test_data.take(STEPS_PER_EPOCH)

In [ ]:
image_batch, label_batch = next(iter(train_data))

### 使用 Keras 训练循环

如果您不需要对训练过程进行低级控制，建议使用 Keras 的内置 `fit`、`evaluate` 和 `predict` 方法。无论实现方式（顺序、函数或子类化）如何，这些方法都能提供统一的接口来训练模型。

这些方法的优点包括：

- 接受 Numpy 数组、Python 生成器和 `tf.data.Datasets`。
- 自动应用正则化和激活损失。
- 支持 `tf.distribute` [ 以进行多设备训练](distributed_training.ipynb)。
- 支持将任意可调用对象作为损失和指标。
- 支持 `tf.keras.callbacks.TensorBoard` 之类的回调以及自定义回调。
- 性能出色，可自动使用 TensorFlow 计算图。

下面是使用 `Dataset` 训练模型的示例（要详细了解工作原理，请参阅[教程](../tutorials)部分）。

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10)
])

# Model is the full model w/o custom layers
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, epochs=NUM_EPOCHS)
loss, acc = model.evaluate(test_data)

print("Loss {}, Accuracy {}".format(loss, acc))

### 编写您自己的循环

如果 Keras 模型的训练步骤适合您，但您需要在该步骤外进行更多控制，则可以考虑在自己的数据迭代循环中使用 `tf.keras.Model.train_on_batch` 方法。

请记住：许多元素可以作为 `tf.keras.callbacks.Callback` 实现。

此方法不仅具备上一部分中提到的方法的诸多优势，而且可以让用户控制外层循环。

您也可以在训练期间使用 `tf.keras.Model.test_on_batch` 或 `tf.keras.Model.evaluate` 来检查性能。

注：`train_on_batch` 和 `test_on_batch` 默认返回单个批次的损失和指标。如果您传递 `reset_metrics=False`，则会返回累加指标，而且必须记得适当地重置指标累加器。另外请记住，像 `AUC` 一类的指标要求正确计算 `reset_metrics=False`。

继续训练上面的模型：


In [ ]:
# Model is the full model w/o custom layers
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

for epoch in range(NUM_EPOCHS):
  #Reset the metric accumulators
  model.reset_metrics()

  for image_batch, label_batch in train_data:
    result = model.train_on_batch(image_batch, label_batch)
    metrics_names = model.metrics_names
    print("train: ",
          "{}: {:.3f}".format(metrics_names[0], result[0]),
          "{}: {:.3f}".format(metrics_names[1], result[1]))
  for image_batch, label_batch in test_data:
    result = model.test_on_batch(image_batch, label_batch,
                                 # return accumulated metrics
                                 reset_metrics=False)
  metrics_names = model.metrics_names
  print("\neval: ",
        "{}: {:.3f}".format(metrics_names[0], result[0]),
        "{}: {:.3f}".format(metrics_names[1], result[1]))

<a name="custom_loop"></a>

### 自定义训练步骤

如果您需要更高的灵活性和更多控制，则可以通过实现自己的训练循环来达到目的。分为三个步骤：

1. 迭代 Python 生成器或 `tf.data.Dataset` 来获得批量样本。
2. 使用 `tf.GradientTape` 收集梯度。
3. 使用 `tf.keras.optimizers` 之一将权重更新应用于模型的变量。

请记住：

- 始终在子类化层和模型的 `call` 方法上包含一个 `training` 参数。
- 确保在 `training` 参数正确设置的情况下调用模型。
- 根据用法，在对一批数据运行模型之前，模型变量可能不存在。
- 您需要手动处理模型的正则化损失等问题。

请注意相对于 v1 的简化：

- 无需运行变量初始值设定项。变量创建时已初始化。
- 无需添加手动控制依赖项。即使在 `tf.function` 中，运算也像在 Eager 模式下一样。

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10)
])

optimizer = tf.keras.optimizers.Adam(0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training=True)
    regularization_loss=tf.math.add_n(model.losses)
    pred_loss=loss_fn(labels, predictions)
    total_loss=pred_loss + regularization_loss

  gradients = tape.gradient(total_loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

for epoch in range(NUM_EPOCHS):
  for inputs, labels in train_data:
    train_step(inputs, labels)
  print("Finished epoch", epoch)


### 新型指标和损失

在 TensorFlow 2.x 中，指标和损失均为对象。两者都在 Eager 模式下工作，且都位于 `tf.function` 中。

损失对象是可调用的，并使用 (y_true, y_pred) 作为参数：


In [ ]:
cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
cce([[1, 0]], [[-1.0,3.0]]).numpy()

指标对象具有以下方法：

- `Metric.update_state()`：添加新观察。
- `Metric.result()`：给定观察值，获取指标的当前结果。
- `Metric.reset_states()`：清除所有观察。

对象本身是可调用的。与 `update_state` 一样，调用会用新的观察更新状态，并返回指标的新结果。

您不必手动初始化指标的变量，而且由于 TensorFlow 2.0 具有自动控制依赖项，因此也不需要担心这两点。

下面的代码使用指标来跟踪自定义训练循环中观察到的平均损失。

In [ ]:
# Create the metrics
loss_metric = tf.keras.metrics.Mean(name='train_loss')
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training=True)
    regularization_loss=tf.math.add_n(model.losses)
    pred_loss=loss_fn(labels, predictions)
    total_loss=pred_loss + regularization_loss

  gradients = tape.gradient(total_loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  # Update the metrics
  loss_metric.update_state(total_loss)
  accuracy_metric.update_state(labels, predictions)


for epoch in range(NUM_EPOCHS):
  # Reset the metrics
  loss_metric.reset_states()
  accuracy_metric.reset_states()

  for inputs, labels in train_data:
    train_step(inputs, labels)
  # Get the metric results
  mean_loss=loss_metric.result()
  mean_accuracy = accuracy_metric.result()

  print('Epoch: ', epoch)
  print('  loss:     {:.3f}'.format(mean_loss))
  print('  accuracy: {:.3f}'.format(mean_accuracy))


<a id="keras_metric_names"></a>

### Keras 指标名称

在 TensorFlow 2.x 中，Keras 模型在处理指标名称方面更加一致。

现在，当您在指标列表中传递字符串时，该*确切*字符串会用作指标的 `name`。这些名称在 `model.fit` 返回的历史对象中可见，而在传递给 `keras.callbacks` 的日志中，它们被设置为您在指标列表中传递的字符串。 

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc', 'accuracy', tf.keras.metrics.SparseCategoricalAccuracy(name="my_accuracy")])
history = model.fit(train_data)

In [ ]:
history.history.keys()

这与以前的版本不同，在以前的版本中，传递 `metrics=["accuracy"]` 会导致 `dict_keys(['loss', 'acc'])` 

### Keras 优化器

`v1.train` 中的优化器（如 `v1.train.AdamOptimizer` 和 `v1.train.GradientDescentOptimizer`）在 `tf.keras.optimizers` 中有对应项。

#### 将 `v1.train` 转换为 `keras.optimizers`

转换优化器时，请注意以下几点：

- 升级优化器[可能会使旧的检查点不兼容](#checkpoints)。
- 现在，所有 ε 的默认值为 `1e-7` 而不是 `1e-8`（大多数情况下可以忽略不计）。
- `v1.train.GradientDescentOptimizer` 可以直接替换为 `tf.keras.optimizers.SGD`。
- 可以通过使用下列动量参数将 `v1.train.MomentumOptimizer` 直接替换为 `SGD` 优化器：`tf.keras.optimizers.SGD(..., momentum=...)`。
- `v1.train.AdamOptimizer` 可在经过转换后使用 `tf.keras.optimizers.Adam`。`beta1` 和 `beta2` 参数已重命名为 `beta_1` 和 `beta_2`。
- `v1.train.RMSPropOptimizer` 可以转换为 `tf.keras.optimizers.RMSprop`。`decay` 参数已重命名为 `rho`。
- `v1.train.AdadeltaOptimizer` 可以直接转换为 `tf.keras.optimizers.Adadelta`。
- `tf.train.AdagradOptimizer` 可以直接转换为 `tf.keras.optimizers.Adagrad`。
- `tf.train.FtrlOptimizer` 可以直接转换为 `tf.keras.optimizers.Ftrl`。`accum_name` 和 `linear_name` 参数已移除。
- `tf.contrib.AdamaxOptimizer` 和 `tf.contrib.NadamOptimizer` 可以直接转换为 `tf.keras.optimizers.Adamax` 和 `tf.keras.optimizers.Nadam`。`beta1` 和 `beta2` 参数已重命名为 `beta_1` and `beta_2`。


#### 某些 `tf.keras.optimizers` 的新默认值

<a id="keras_optimizer_lr"></a>

警告：如果您发现模型的收敛行为发生变化，请检查默认学习率。

`optimizers.SGD`、`optimizers.Adam` 和 `optimizers.RMSprop` 没有任何变更。

以下优化器的默认学习率已更改：

- `optimizers.Adagrad` 从 0.01 更改为 0.001
- `optimizers.Adadelta` 从 1.0 更改为 0.001
- `optimizers.Adamax` 从 0.002 更改为 0.001
- `optimizers.Nadam` 从 0.002 更改为 0.001

### TensorBoard

TensorFlow 2.x 包含 `tf.summary` API 的重大变更，该 API 用于写入摘要数据以在 TensorBoard 中进行可视化。有关全新 `tf.summary` 的总体介绍，请参阅使用 TensorFlow 2.x API 的[多个教程](https://tensorflow.google.cn/tensorboard/get_started)，其中包括 [ TensorBoard TensorFlow 2.x 迁移指南](https://tensorflow.google.cn/tensorboard/migrate)。

## 保存和加载


<a id="checkpoints"></a>

### 检查点兼容性

TensorFlow 2.x 使用[基于对象的检查点](checkpoint.ipynb)。

如果您细心一点，仍然可以加载基于名称的旧式检查点。代码转换过程可能会导致变量名更改，但是有变通方式。

最简单的方式是将新模型的名称与检查点中的名称对齐：

- 变量仍然具有可以设置的 `name` 参数。
- Keras 模型还采用 `name` 参数，并将其设置为变量的前缀。
- `v1.name_scope` 函数可用于设置变量名前缀，这与 `tf.variable_scope` 截然不同。它只影响名称，而不跟踪变量和重用。

如果这不适合您的用例，请尝试使用 `v1.train.init_from_checkpoint` 函数。它需要一个 `assignment_map` 参数，该参数指定从旧名称到新名称的映射。

注：与基于对象的检查点（可以[延迟加载](checkpoint.ipynb#loading_mechanics)）不同，基于名称的检查点要求在调用函数时构建所有变量。某些模型会延迟构建变量，直到您调用 `build` 或对一批数据运行模型。

[TensorFlow Estimator 仓库](https://github.com/tensorflow/estimator/blob/master/tensorflow_estimator/python/estimator/tools/checkpoint_converter.py)包含一个[转换工具](#checkpoint_converter)，可将预制 Estimator 的检查点从 TensorFlow 1.X 升级到 2.0。它可以作为如何为类似用例构建工具的示例。

### 已保存模型的兼容性

对于已保存的模型，没有明显的兼容性问题：

- TensorFlow 1.x saved_models 可在 TensorFlow 2.x 中运行。
- 如果所有运算均受到支持，TensorFlow 2.x saved_models 甚至可以在 TensorFlow 1.x 中运行。

### Graph.pb 或 Graph.pbtxt 

无法直接将原始 `Graph.pb` 文件升级到 TensorFlow 2.x。最佳选择是升级生成文件的代码。

但是，如果您有一个“冻结计算图”（其中的变量已被转换为常量的 `tf.Graph`），则可以使用 <code>v1.wrap_function</code> 将其转换为 <a><code data-md-type="codespan">concrete_function</code></a>：


In [ ]:
def wrap_frozen_graph(graph_def, inputs, outputs):
  def _imports_graph_def():
    tf.compat.v1.import_graph_def(graph_def, name="")
  wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
  import_graph = wrapped_import.graph
  return wrapped_import.prune(
      tf.nest.map_structure(import_graph.as_graph_element, inputs),
      tf.nest.map_structure(import_graph.as_graph_element, outputs))

例如，下面是 2016 年的 Inception v1 的冻结计算图：

In [ ]:
path = tf.keras.utils.get_file(
    'inception_v1_2016_08_28_frozen.pb',
    'http://storage.googleapis.com/download.tensorflow.org/models/inception_v1_2016_08_28_frozen.pb.tar.gz',
    untar=True)

加载 `tf.GraphDef`：

In [ ]:
graph_def = tf.compat.v1.GraphDef()
loaded = graph_def.ParseFromString(open(path,'rb').read())

将其封装到 `concrete_function` 中：

In [ ]:
inception_func = wrap_frozen_graph(
    graph_def, inputs='input:0',
    outputs='InceptionV1/InceptionV1/Mixed_3b/Branch_1/Conv2d_0a_1x1/Relu:0')

将张量作为输入传递给它：

In [ ]:
input_img = tf.ones([1,224,224,3], dtype=tf.float32)
inception_func(input_img).shape

## Estimator

### 使用 Estimator 进行训练

TensorFlow 2.x 支持 Estimator。

使用 Estimator 时，您可以使用 TensorFlow 1.x 中的 `input_fn()`、`tf.estimator.TrainSpec` 和 `tf.estimator.EvalSpec`。

下面是将 `input_fn` 与训练和评估规范结合使用的示例：

#### 创建 input_fn 和 train/eval 规范

In [ ]:
# Define the estimator's input_fn
def input_fn():
  datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_train, mnist_test = datasets['train'], datasets['test']

  BUFFER_SIZE = 10000
  BATCH_SIZE = 64

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label[..., tf.newaxis]

  train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  return train_data.repeat()

# Define train &amp; eval specs
train_spec = tf.estimator.TrainSpec(input_fn=input_fn,
                                    max_steps=STEPS_PER_EPOCH * NUM_EPOCHS)
eval_spec = tf.estimator.EvalSpec(input_fn=input_fn,
                                  steps=STEPS_PER_EPOCH)


### 使用 Keras 模型定义

在 TensorFlow2.x 中如何构造 Estimator 存在一些差异。

建议您使用 Keras 定义模型，然后使用 `tf.keras.estimator.model_to_estimator` 实用工具将您的模型转换为 Estimator。下面的代码展示了如何在创建和训练 Estimator 时使用此实用工具。

In [ ]:
def make_model():
  return tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10)
  ])

In [ ]:
model = make_model()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(
  keras_model = model
)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

注：TensorFlow 不支持使用 `model_to_estimator` 在 Keras 中创建加权指标并将其转换为 Estimator API 中的加权指标。您必须使用 `add_metrics` 函数直接在 Estimator 规范上创建这些指标。

### 使用自定义 `model_fn`

如果您需要维护现有的自定义 Estimator `model_fn`，则可以将 `model_fn` 转换为使用 Keras 模型。

但是，出于兼容性原因，自定义 `model_fn` 仍将在 1.x 样式的计算图模式下运行，这意味着既没有 Eager Execution，也没有自动控制依赖项。

注：从长远来看，您应当计划从 `tf.estimator` 迁移，特别是使用自定义 `model_fn`。替代 API 是 `tf.keras` 和 `tf.distribute`。如果您的训练的某个部分仍需要 `Estimator`，您可以使用 `tf.keras.estimator.model_to_estimator` 转换器从 `keras.Model` 创建 `Estimator`。


<a name="minimal_changes"></a>

#### 细微改动的自定义 model_fn

为了使您的自定义 `model_fn` 在 TensorFlow 2.x 中运行，如果您希望对现有代码进行细微改动，可以使用 `tf.compat.v1` 符号，例如 `optimizers` 和 `metrics`。

在自定义 `model_fn` 中使用 Keras 模型与在自定义训练循环中使用它类似：

- 根据 `mode` 参数适当设置 `training` 阶段。
- 将模型的 `trainable_variables` 显式传递给优化器。

但是，相对于[自定义循环](#custom_loop)，存在重要差异：

- 使用 `Model.get_losses_for` 提取损失，而不是使用 `Model.losses`。
- 使用 `Model.get_updates_for` 提取模型的更新。

注：“更新”是每个批次后需要应用于模型的变更。例如， `layers.BatchNormalization` 层中均值和方差的移动平均值。

以下代码可从自定义 `model_fn` 创建一个 Estimator，说明了所有这些问题。

In [ ]:
def my_model_fn(features, labels, mode):
  model = make_model()

  optimizer = tf.compat.v1.train.AdamOptimizer()
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

  training = (mode == tf.estimator.ModeKeys.TRAIN)
  predictions = model(features, training=training)

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  reg_losses = model.get_losses_for(None) + model.get_losses_for(features)
  total_loss=loss_fn(labels, predictions) + tf.math.add_n(reg_losses)

  accuracy = tf.compat.v1.metrics.accuracy(labels=labels,
                                           predictions=tf.math.argmax(predictions, axis=1),
                                           name='acc_op')

  update_ops = model.get_updates_for(None) + model.get_updates_for(features)
  minimize_op = optimizer.minimize(
      total_loss,
      var_list=model.trainable_variables,
      global_step=tf.compat.v1.train.get_or_create_global_step())
  train_op = tf.group(minimize_op, update_ops)

  return tf.estimator.EstimatorSpec(
    mode=mode,
    predictions=predictions,
    loss=total_loss,
    train_op=train_op, eval_metric_ops={'accuracy': accuracy})

# Create the Estimator &amp; Train
estimator = tf.estimator.Estimator(model_fn=my_model_fn)
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

#### 带 TensorFlow 2.x 符号的自定义 `model_fn`

如果要除去所有 TensorFlow 1.x 符号并将自定义 `model_fn` 升级到 TensorFlow 2.x，则需要将优化器和指标更新为 `tf.keras.optimizers` 和 `tf.keras.metrics`。

在自定义 `model_fn` 中，除了上述[变更](#minimal_changes)外，还需要进行更多升级：

- 使用 [`tf.keras.optimizers`](https://tensorflow.google.cn/versions/r2.0/api_docs/python/tf/keras/optimizers) 代替 `v1.train.Optimizer`。
- 将模型的 `trainable_variables` 显式传递给 `tf.keras.optimizers`。
- 要计算 `train_op/minimize_op`，请执行以下操作：
    - 如果损失是标量损失 `Tensor`（不是可调用对象），则使用 `Optimizer.get_updates`。返回列表中的第一个元素是所需的 `train_op/minimize_op`。
    - 如果损失是可调用对象（例如函数），则使用 `Optimizer.minimize` 来获取 `train_op/minimize_op`。
- 使用 [`tf.keras.metrics`](https://tensorflow.google.cn/api_docs/python/tf/keras/metrics) 代替 `tf.compat.v1.metrics` 进行评估。

对于上面的 `my_model_fn` 示例，包含 TensorFlow 2.x 符号的已迁移代码如下所示：

In [ ]:
def my_model_fn(features, labels, mode):
  model = make_model()

  training = (mode == tf.estimator.ModeKeys.TRAIN)
  loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  predictions = model(features, training=training)

  # Get both the unconditional losses (the None part)
  # and the input-conditional losses (the features part).
  reg_losses = model.get_losses_for(None) + model.get_losses_for(features)
  total_loss=loss_obj(labels, predictions) + tf.math.add_n(reg_losses)

  # Upgrade to tf.keras.metrics.
  accuracy_obj = tf.keras.metrics.Accuracy(name='acc_obj')
  accuracy = accuracy_obj.update_state(
      y_true=labels, y_pred=tf.math.argmax(predictions, axis=1))

  train_op = None
  if training:
    # Upgrade to tf.keras.optimizers.
    optimizer = tf.keras.optimizers.Adam()
    # Manually assign tf.compat.v1.global_step variable to optimizer.iterations
    # to make tf.compat.v1.train.global_step increased correctly.
    # This assignment is a must for any `tf.train.SessionRunHook` specified in
    # estimator, as SessionRunHooks rely on global step.
    optimizer.iterations = tf.compat.v1.train.get_or_create_global_step()
    # Get both the unconditional updates (the None part)
    # and the input-conditional updates (the features part).
    update_ops = model.get_updates_for(None) + model.get_updates_for(features)
    # Compute the minimize_op.
    minimize_op = optimizer.get_updates(
        total_loss,
        model.trainable_variables)[0]
    train_op = tf.group(minimize_op, *update_ops)

  return tf.estimator.EstimatorSpec(
    mode=mode,
    predictions=predictions,
    loss=total_loss,
    train_op=train_op,
    eval_metric_ops={'Accuracy': accuracy_obj})

# Create the Estimator &amp; Train.
estimator = tf.estimator.Estimator(model_fn=my_model_fn)
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

### 预制 Estimator

TensorFlow 2.x API 仍支持 <code>tf.estimator.DNN*</code>、`tf.estimator.Linear*` 和 `tf.estimator.DNNLinearCombined*` 系列中的<a>预制 Estimator</a>，但是一些参数已更改：

1. `input_layer_partitioner`：在 v2 中移除。
2. `loss_reduction`：更新为 `tf.keras.losses.Reduction`，代替 `tf.compat.v1.losses.Reduction`。其默认值也从 `tf.compat.v1.losses.Reduction.SUM` 更改为 `tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE`。
3. `optimizer`、`dnn_optimizer` 和 `linear_optimizer`：此参数已更新为 `tf.keras.optimizers`，代替 `tf.compat.v1.train.Optimizer`。

要迁移上述变更，请执行以下操作：

1. `input_layer_partitioner` 不需要迁移，因为 [`Distribution Strategy`](https://tensorflow.google.cn/guide/distributed_training) 将在 TensorFlow 2.x 中自动处理它。
2. 对于 `loss_reduction`，检查 [`tf.keras.losses.Reduction`](https://tensorflow.google.cn/versions/r2.0/api_docs/python/tf/keras/losses/Reduction) 是否存在支持的选项。
3. 对于 `optimizer` 参数：
    - 如果您不：1) 传入 `optimizer`、`dnn_optimizer` 或 `linear_optimizer` 参数，或者 2) 在代码中作为 `string` 指定 `optimizer` 参数，则不需要进行任何更改，因为会默认使用 `tf.keras.optimizers`。
    - 否则，您需要将其从 `tf.compat.v1.train.Optimizer` 更新为对应的 `tf.keras.optimizers`。


#### 检查点转换器

<a id="checkpoint_converter"></a>

迁移到 `keras.optimizers` 将破坏使用 TensorFlow 1.x 保存的检查点，因为 `tf.keras.optimizers` 会生成一组不同的变量以保存在检查点中。要在迁移到 TensorFlow 2.x 后使旧的检查点可重用，请尝试使用[检查点转换器工具](https://github.com/tensorflow/estimator/blob/master/tensorflow_estimator/python/estimator/tools/checkpoint_converter.py)。

In [ ]:
! curl -O https://raw.githubusercontent.com/tensorflow/estimator/master/tensorflow_estimator/python/estimator/tools/checkpoint_converter.py

该工具具有内置帮助：

In [ ]:
! python checkpoint_converter.py -h

<a id="tensorshape"></a>

## TensorShape

此类已经过简化，可以保存 `int` 而不是 `tf.compat.v1.Dimension` 对象。因此，无需调用 `.value` 来获取 `int`。

仍然可以从 `tf.TensorShape.dims` 访问各个 `tf.compat.v1.Dimension` 对象。

以下代码演示了 TensorFlow 1.x 和 TensorFlow 2.x 之间的区别。

In [ ]:
# Create a shape and choose an index
i = 0
shape = tf.TensorShape([16, None, 256])
shape

如果您在 TensorFlow 1.x 中运行此代码：

```python
value = shape[i].value
```

则在 TensorFlow 2.x 中运行此代码：


In [ ]:
value = shape[i]
value

如果您在 TensorFlow 1.x 中运行此代码：

```python
for dim in shape:     value = dim.value     print(value)
```

则在 TensorFlow 2.x 中运行此代码：

In [ ]:
for value in shape:
  print(value)

如果您在 TensorFlow 1.x 中（或使用任何其他维度方法）运行此代码：

```python
dim = shape[i] dim.assert_is_compatible_with(other_dim)
```

则在 TensorFlow 2.x 中运行此代码：

In [ ]:
other_dim = 16
Dimension = tf.compat.v1.Dimension

if shape.rank is None:
  dim = Dimension(None)
else:
  dim = shape.dims[i]
dim.is_compatible_with(other_dim) # or any other dimension method

In [ ]:
shape = tf.TensorShape(None)

if shape:
  dim = shape.dims[i]
  dim.is_compatible_with(other_dim) # or any other dimension method

如果秩已知，`tf.TensorShape` 的布尔值将为 `True`，否则为 `False`。

In [ ]:
print(bool(tf.TensorShape([])))      # Scalar
print(bool(tf.TensorShape([0])))     # 0-length vector
print(bool(tf.TensorShape([1])))     # 1-length vector
print(bool(tf.TensorShape([None])))  # Unknown-length vector
print(bool(tf.TensorShape([1, 10, 100])))       # 3D tensor
print(bool(tf.TensorShape([None, None, None]))) # 3D tensor with no known dimensions
print()
print(bool(tf.TensorShape(None)))  # A tensor with unknown rank.

## 其他变更

- 移除 `tf.colocate_with`：TensorFlow 的设备放置算法已得到显著改善。它不再是必需的。如果移除它导致性能下降，请[提交错误报告](https://github.com/tensorflow/tensorflow/issues)。

- 将 `v1.ConfigProto` 用法替换为 `tf.config` 中的等效函数。


## 结论

整个流程为：

1. 运行升级脚本。
2. 移除 contrib 符号。
3. 将模型切换为面向对象的样式 (Keras)。
4. 尽可能使用 `tf.keras` 或 `tf.estimator` 训练和评估循环。
5. 否则，请使用自定义循环，但需确保避免会话和集合。

将代码转换为惯用的 TensorFlow 2.x 需要一些工作，但每次更改都会带来下列好处：

- 减少代码行。
- 提高清晰度和简洁性。
- 简化调试。